### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth or np.iinfo(np.int32).max
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return np.square(l_c).sum(axis=1) / l_s +\
            np.square(r_c).sum(axis=1) / r_s -\
            np.square(l_c + r_c).sum(axis=1) / (l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return (np.log2(l_c / np.expand_dims(l_s,
                                             axis=1)) * l_c).sum(axis=1) +\
            (np.log2(r_c / np.expand_dims(r_s,
                                          axis=1)) * r_c).sum(axis=1) -\
            (np.log2((l_c + r_c) / np.expand_dims((l_s + r_s),
                                                  axis=1)) *
             (l_c + r_c)).sum(axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_c.max() + r_c.max() - (l_c + r_c).max()

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: max(1, int(np.sqrt(n_feature)))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: max(1, int(np.log2(n_feature)))]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        th_idx = np.where(y[1:] != y[:-1])[0] + 1
        if not th_idx.any():
            return [0, 0]
        seq = th_idx - np.append([0], th_idx[:-1])
        change_cl = np.zeros((th_idx.shape[0], self.num_class))
        change_cl[np.arange(th_idx.shape[0]), y[th_idx - 1]] = 1
        class_counts = change_cl * seq.reshape(-1, 1)
        r_c_ = np.cumsum(class_counts, axis=0)
        l_c_ = np.bincount(y, minlength=self.num_class) - r_c_
        epsilon = 0.001
        r_c_[r_c_ == 0] = epsilon
        l_c_[l_c_ == 0] = epsilon
        l_s_ = l_c_.astype('int').sum(axis=1)
        r_s_ = y.shape[0] - l_s_
        imp_mat = self.G_function(l_c_, l_s_, r_c_, r_s_)
        return [imp_mat.max(), x[r_s_[imp_mat.argmax()] - 1]]

    def __stop_criterion(self, cur_depth, samples, max_proba):
        depth = self.max_depth > cur_depth
        sample = self.min_samples_split < samples
        share = self.sufficient_share > max_proba
        if depth and sample and share:
            return False
        else:
            return True

    def __fit_node(self, x, y, node_id, depth):
        y_proba = np.bincount(y, minlength=self.num_class) / y.shape[0]
        if self.__stop_criterion(depth, y.shape[0], y_proba.max()):
#           y_pred = self.classes[y_proba.argmax()]
            y_pred = y_proba.argmax()
            self.tree.update({node_id: (self.__class__.LEAF_TYPE,
                                        y_pred, y_proba)})
        else:
            f_ids = self.get_feature_ids(x.shape[1])
            node_g = np.array([])
            node_th = np.array([])
            for f_i in f_ids:
                x_i, y_i = self.__sort_samples(x[:, f_i], y)
                g, th = self.__find_threshold(x_i, y_i)
                node_g = np.append(node_g, g)
                node_th = np.append(node_th, th)
            best = node_g.argmax()
            best_f_i = f_ids[best]
            best_threashold = node_th[best]
            self.feature_importances_[best] += node_g.max()
            x_l, x_r, y_l, y_r = self.__div_samples(x,
                                                    y,
                                                    best_f_i,
                                                    best_threashold)
            if len(y_l) == 0 or len(y_r) == 0:
                y_proba = np.bincount(y, minlength=self.num_class) / y.shape[0]
#                 y_pred = self.classes[y_proba.argmax()]
                y_pred = y_proba.argmax()
                self.tree.update({node_id: (self.__class__.LEAF_TYPE,
                                            y_pred, y_proba)})
                return
            self.tree.update({node_id: (self.__class__.NON_LEAF_TYPE,
                                        best_f_i,
                                        best_threashold)})
#             print('tree: ', self.tree[node_id])
            self.__fit_node(x_l, y_l, node_id * 2 + 1, depth + 1)
            self.__fit_node(x_r, y_r, node_id * 2 + 2, depth + 1)

    def fit(self, x, y):
#         self.classes = np.unique(y)
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.shape[0]
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [29]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [30]:
wine = load_wine()
X_train, X_test, y_train, y_test =\
    train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [31]:
%time clf.fit(X_train, y_train)

CPU times: user 2.24 ms, sys: 1.39 ms, total: 3.64 ms
Wall time: 2.02 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
%time my_clf.fit(X_train, y_train)

CPU times: user 23 ms, sys: 3.29 ms, total: 26.3 ms
Wall time: 23.8 ms


## Проверка качества работы на wine

In [33]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8363636363636364

In [34]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8363636363636364

## Подготовка данных Speed Dating Data 

In [35]:
df = pd.read_csv('./data/speed-dating-experiment/Speed Dating Data.csv',
                 encoding='latin1')
df = df.iloc[:, :97]

In [36]:
#%%pycodestyle


df = df.drop(['id', 'idg', 'condtn', 'position',
              'positin1', 'order', 'partner',
              'age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int', 'pf_o_fun',
              'pf_o_amb', 'pf_o_sha', 'dec_o',
              'attr_o', 'sinc_o', 'intel_o',
              'fun_o', 'amb_o', 'shar_o', 'like_o',
              'prob_o','met_o', 'field', 'undergra',
              'from', 'zipcode', 'income', 'sports',
              'tvsports','exercise','dining', 'round',
              'museums','art','hiking','gaming',
              'clubbing','reading','tv','theater',
              'movies','concerts','music','shopping',
              'yoga', 'expnum'], axis=1)

df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df.field_cd = df.field_cd.astype(np.int)

df.race = df.race.astype(np.int)

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)
df.career_c = df.career_c.astype(np.int)

# df.loc[:, 'mn_sat'] =\
#     df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
# df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].fillna(0)
df = df.drop(['mn_sat'], axis=1)

# df.loc[:, 'tuition'] =\
#     df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
# df.loc[:, 'tuition'] = df.loc[:, 'tuition'].fillna(0)
df = df.drop(['tuition'], axis=1)

df = df.dropna(subset=['imprelig', 'imprace', 'date'])

In [37]:
df.loc[:, 'temp_totalsum'] =\
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
               'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                'fun1_1', 'amb1_1', 'shar1_1']].T/
     df.loc[:, 'temp_totalsum'].T).T * 100

In [38]:
df.loc[:, 'temp_totalsum'] =\
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
               'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
           'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                'fun2_1', 'amb2_1', 'shar2_1']].T/
     df.loc[:, 'temp_totalsum'].T).T * 100

In [39]:
df = df.drop(['temp_totalsum'], axis=1)

In [40]:
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)

In [41]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr',
                                          'samerace'], axis=1).dropna()
        
df_female.columns = df_female.columns + '_f'

df_male.pid = df_male.pid.astype(np.int)
df_female.pid_f = df_female.pid_f.astype(np.int)

In [42]:
df_join = df_male.join(df_female.set_index('iid_f'), how='right', on='pid')

In [43]:
target = df_join.loc[:, 'match'].values
df_join = df_join.drop(['match', 'iid', 'pid', 'pid_f'], axis=1)
data = df_join.values

print(df_join.head())

     int_corr  samerace   age  field_cd  race  imprace  imprelig  goal  date  \
100      0.14         0  27.0         8     2      7.0       3.0   1.0   5.0   
110      0.54         0  22.0         1     2      1.0       1.0   1.0   1.0   
120      0.16         1  22.0         1     4      3.0       5.0   2.0   7.0   
130      0.61         0  23.0         1     2      1.0       1.0   2.0   4.0   
140      0.21         0  24.0         1     3      3.0       1.0   1.0   4.0   

     go_out    ...     sinc2_1_f  intel2_1_f  fun2_1_f  amb2_1_f  shar2_1_f  \
100     4.0    ...          20.0        15.0      20.0       5.0        5.0   
110     1.0    ...          20.0        15.0      20.0       5.0        5.0   
120     1.0    ...          20.0        15.0      20.0       5.0        5.0   
130     1.0    ...          20.0        15.0      20.0       5.0        5.0   
140     1.0    ...          20.0        15.0      20.0       5.0        5.0   

     attr3_1_f  sinc3_1_f  fun3_1_f  intel3_

In [54]:
SDD_my_clf = MyDecisionTreeClassifier(min_samples_split=2)
SDD_clf = DecisionTreeClassifier(min_samples_split=2)

In [55]:
X_train_SDD, X_test_SDD, y_train_SDD, y_test_SDD = train_test_split(data, target,
                                                    test_size=0.01, stratify=target)

## Проверка скорости работы на Speed Dating Data 

In [56]:
%time SDD_clf.fit(X_train_SDD, y_train_SDD)

CPU times: user 103 ms, sys: 3.75 ms, total: 106 ms
Wall time: 104 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [57]:
%time SDD_my_clf.fit(X_train_SDD, y_train_SDD)

CPU times: user 1.19 s, sys: 8.69 ms, total: 1.19 s
Wall time: 1.19 s


## Проверка качества работы на Speed Dating Data

In [58]:
f1_score(y_pred=SDD_clf.predict(X_test_SDD), y_true=y_test_SDD, average='macro')

0.4029850746268657

In [59]:
f1_score(y_pred=SDD_my_clf.predict(X_test_SDD), y_true=y_test_SDD, average='macro')

0.4444444444444444

## Задание 3

In [64]:
# Добавьте функционал, который определяет значения feature importance.

print(SDD_clf.feature_importances_[:10], ', ...')
print(SDD_my_clf.feature_importances_[:10], ', ...')

[0.08083297 0.00681826 0.02850256 0.02182158 0.00793687 0.0187251
 0.01082891 0.01260096 0.02032261 0.01647445] , ...
[0.03708706 0.03685213 0.02867055 0.03263559 0.02183774 0.05207775
 0.01090708 0.01734712 0.03868061 0.025555  ] , ...


## Задание 4

In [76]:
# Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже)
# для MyDecisionTreeClassifier и DecisionTreeClassifier так,
# чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier.
# Используем данные Speed Dating Data.

f_idx = np.argsort(SDD_clf.feature_importances_)[::-1]
my_f_idx = np.argsort(SDD_my_clf.feature_importances_)[::-1]

features = zip(df_join.columns[f_idx][:10],
           SDD_clf.feature_importances_[f_idx][:10])
my_features = zip(df_join.columns[my_f_idx][:10],
                  SDD_clf.feature_importances_[my_f_idx][:10])
# features


print('Sklearn DecisionTreeClassifier:\n', *features, sep='\n')
print('\nMyDecisionTreeClassifier:\n', *my_features, sep='\n')

Sklearn DecisionTreeClassifier:

('int_corr', 0.08083297082910103)
('amb3_1_f', 0.0408342304573731)
('exphappy_f', 0.030387920332305574)
('fun2_1_f', 0.02907707013181051)
('fun1_1_f', 0.028970194144291805)
('career_c_f', 0.028572191548502636)
('age', 0.028502555759906564)
('age_f', 0.027017507458596053)
('sinc1_1', 0.02467887822865914)
('intel1_1_f', 0.023549510546904996)

MyDecisionTreeClassifier:

('age_f', 0.027017507458596053)
('imprace', 0.018725099198650015)
('fun3_1', 0.002861581110448058)
('intel1_1', 0.020568612630350827)
('date', 0.02032261222201473)
('career_c', 0.009476480998626547)
('int_corr', 0.08083297082910103)
('exphappy', 0.015643570085250635)
('samerace', 0.0068182559593587076)
('shar1_1', 0.008158511763778435)


## Задание 5

In [66]:
# С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса
# (Выберете 2-3 параметра).
# Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

from scipy.stats import randint as randint


param_grid = {
#     'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 8),
    'min_samples_split': randint(5, 10),
    'class_weight': [None, 'balanced']}

cv = KFold(n_splits=5, random_state=123, shuffle=True)

# model = DecisionTreeClassifier(random_state=123)

model_RF = RandomForestClassifier(random_state=123,
                                  n_estimators=50,
                                  n_jobs=-1)

random_search = RandomizedSearchCV(model_RF, param_distributions=param_grid,
                                   n_iter=200, n_jobs=-1, cv=cv,
                                   scoring='roc_auc', random_state=123)
# А дальше, просто .fit()
random_search.fit(data, target)

print(random_search.best_params_)

print(random_search.best_score_)

# best_model = random_search.best_estimator_

{'class_weight': 'balanced', 'max_depth': 7, 'min_samples_split': 8}
0.6894952574263359
